In [ ]:
# use tf dataset api to load dataset 
# prior to this , use downloadv3.sh and post_download.sh


# self notes
# custom function must strictly pass numpy dtype as input and numpy dtype as output ( not tensors)

In [26]:
import tensorflow  as tf

In [38]:
import sys

sys.path.insert(0, '/media/saurabh/New Volume/tf_audio_sentiment/pyAudioAnalysis')

from  pyAudioAnalysis  import audioBasicIO
from  pyAudioAnalysis import audioFeatureExtraction

In [43]:
timesteps=2000 # max timesteps
data_base_directory='/media/saurabh/New Volume/tf_audio_sentiment/data_test/datav2/'

In [44]:
def audio_to_features(fileurl):
    
    [Fs, x] = audioBasicIO.readAudioFile(fileurl);
    F = audioFeatureExtraction.stFeatureExtraction(x, Fs, 0.050*Fs, 0.025*Fs);
    
    a=F[:21,:]
    b=F[33]

    b=np.expand_dims(b, axis=0)

    #print(b.shape)


    d = np.concatenate( (a, b), axis=0)
    #print(d.shape)
    d = np.swapaxes(d, 0, 1)
    return d

In [45]:
# define a function, that pads audio so that audio frames = max frames

def pad(input):
   # print(input.shape[0])
     if input.shape[0] < timesteps:
        
        diff = timesteps - input.shape[0]
        
        # pad and return input
        return np.pad(input,((0,diff),(0,0)), mode="constant")
    
     elif input.shape[0] > timesteps:
        
        return input[:timesteps,:]

In [46]:
def process_function(input):
  
  #input_string= tf.convert_to_tensor(input, dtype=tf.string)
  y = 0

  input=pad(audio_to_features(str(input)))

  if 'A' in str(input):
        y=0
  elif 'B' in str(input):
       y=1
    
  return  input,y

In [47]:
logfilepath='/media/saurabh/New Volume/tf_audio_sentiment/data_test/dataset.txt'
dataset = tf.contrib.data.TextLineDataset(logfilepath).shuffle(buffer_size=800)#.map(map_function)

dataset = dataset.map(
    lambda filename: tuple(tf.py_func(
        process_function, [filename],  [tf.string,tf.int64])))


In [48]:
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [49]:
with tf.Session() as sess:
    for i in range(1000):
      value = sess.run(next_element)
      print(str(value))

Error in readAudioFile(): Unknown file type!


InvalidArgumentError: TypeError: object of type 'numpy.float64' has no len()
	 [[Node: PyFunc = PyFunc[Tin=[DT_STRING], Tout=[DT_STRING, DT_INT64], token="pyfunc_9"](arg0)]]
	 [[Node: IteratorGetNext_9 = IteratorGetNext[output_shapes=[<unknown>, <unknown>], output_types=[DT_STRING, DT_INT64], _device="/job:localhost/replica:0/task:0/cpu:0"](OneShotIterator_9)]]

<tf.Tensor 'Const:0' shape=() dtype=int32>